## Implementation of a quantum algorithm for Discrete Log Problem over $\mathbb{Z}^*_N$ using IBM QisKit
Here, we present an implementation of a quantum algorithm for the discrete log problem on the multiplicative group. The algorithm is inspired from the explanation in the book **An Introduction to Quantum Computing**  by **P. Kaye, R. Laflamme, M. Mosca**. ([Link](http://mmrc.amss.cas.cn/tlb/201702/W020170224608149125645.pdf)). Refer section 7.4 of the book.

Here is the statement of discrete log problem for a group $G$.
Let $a, b \in G$. Let $b = a^t$. $t$ is called a discrete logarithm of $b$ to the base $a$.  The discrete log problem asks: Given $a, b \in G$, find a discrete logarithm of $b$ to the base $a$ or report that none exists.

There are two main parts of the algorithm for $G=\mathbb{Z}^*_N$ where $N \in \mathbb{Z}^{\geq 2}$:
The first part is the implementation of the transformation for modular multiplication.  For $a \in \mathbb{Z}^*_N$
$$U_a |x\rangle = |ax \text{ mod }N \rangle$$ This implementation is carried out in stages using the following functions: $\texttt{get_angles()}$, $\texttt{phi_add()}$, $\texttt{phi_add_mod_N()}$, $\texttt{multiply_mod_N()}$ and $\texttt{power_mod_N()}$. 

The implementation of the first four functions is based on the [this](https://arxiv.org/abs/quant-ph/0205095) paper by Stephane Beauregard and is heavily inspired by [this](https://qiskit.org/documentation/_modules/qiskit/aqua/algorithms/single_sample/shor/shor.html) Qiskit implementation of Shor's algorithm.

The second part is the implementation of the phase estimation algorithm.
![](https://drive.google.com/uc?id=19izOOlQ7CvGuVlPjRrH3nJ3vpQROon6W)

Using Shor's algorithm we can find $r$, the order of $a$ modulo $N$. WLOG, let $t \in \mathbb{Z}_r$. Now, on measuring the first two registers, we can find $(\frac{k}{r}, \frac{kt \text{ mod }r}{r} )$ for some $k \in \mathbb{Z}_r$. Since we know $r$, we get the pair $(k, kt \text{ mod }r)$. We run the circuit multiple times to find two pairs $(k_1, k_1t \text{ mod }r)$ and $(k_2, k_2t \text{ mod }r)$ such that $k_1$ and $k_2$ are relatively prime. Let $v_1 = k_1t \text{ mod }r$ and $v_2 = k_2t \text{ mod }r$. Now there exist $\lambda_1, \lambda_2 \in \mathbb{Z}$ such that $$\lambda_1 k_1 + \lambda_2 k_2 = 1$$ So, $t =  \lambda_1 k_1 t + \lambda_2 k_2 t$. Since $t = t \text{ mod }r$, $$t = (\lambda_1 v_1 + \lambda_2 v_2) \text{ mod }r$$




In [0]:
!pip install qiskit==0.12.0

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 21.6MB 72.9MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 983kB 45.8MB/s 
     |████████████████████████████████| 5.6MB 42.1MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 573kB 49.7MB/s 
     |████████████████████████████████| 24.3MB 1.8MB/s 
     |████████████████████████████████| 2.7MB 42.9MB/s 
  Created wheel for qiskit: filename=qiskit-0.12.0-cp36-none-any.whl size=1397 sha256=4bf047879260b3bdac57129bbfb3990b236b6fd100b05b5ea31313ba9e50d6ab
  Stored in directory: /root/.cache/pip/wheels/de/fc/68/2318ad43f26583ec1ac8e30bedd43e71211c9256429b7f829c
  Created wheel for dlx: filename=dlx-1.0.4-cp36-none-any.whl size=5712 sha256

In [0]:
import numpy as np
import math
from time import time
from math import gcd
from qiskit import(QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer)
from qiskit.aqua.circuits import FourierTransformCircuits as ftc
from qiskit.aqua.circuits.gates import mcu1
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.utils import summarize_circuits

$\texttt{mod-exp(a, b, N)}$ computes $a^b\text{ mod }N$ where $a \in \mathbb{Z}$, $b \in \mathbb{Z}^{\geq0}$ and $N \in \mathbb{Z}^+$.	

In [0]:
def mod_exp(a, b, N):
    X = int(a)
    E = int(b)
    m = int(N)
    if(E<0 or m<1):
        return None
    X = X % m
    Y = 1
    while E > 0:
        if E % 2 == 0:
            X = (X * X) % m
            E = E/2
        else:
            Y = (X * Y) % m
            E = E - 1
    return Y

$\texttt{egcd(a, b)}$, where  $a, b \in \mathbb{Z}$,  returns a 3-tuple $(g, x, y)$ where  $a, b, g \in \mathbb{Z}$ such that $|g| = \text{gcd}(a, b)$ and $ax + by = g$. If $a=b=0$, the $g=0$ is returned.

In [0]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

$\texttt{modinv(a, n)}$, where $a, n \in \mathbb{Z}$ such that $n \geq 2$, returns $a^{-1} \text{ mod } n$ if $a$ and $n$ are relatively prime and returns $\texttt{None}$ otherwise.

In [0]:
def modinv(a, n):
    if(n<2):
        return None
    g, _, x = egcd(n, a)
    if(g!=1):
        return None
    return x % n

$\texttt{order(a, N)}$, where $a \in \mathbb{Z}$ and $N \in \mathbb{Z}^{\geq 2}$ such that $a$ and $N$ are relatively prime, returns the smallest positive integer $r$ such that $a^r \equiv 1 \text{ mod } N$.  One may replace it with Shor's order finding algorithm.

In [0]:
def order(a, N):
    N = int(N)
    a = int(a) % N
    if(N<2 or gcd(a, N)!=1):
        return None
    r = 1
    p = a
    while(p != 1):
        p = (p*a) % N
        r = r + 1
    return r

$\texttt{get_noise(p_meas, p_gate)}$ sets the noise parameters for gates and measurements. For full explanation, see [this link](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html#Correcting-errors-in-qubits) 

In [0]:
def get_noise(p_meas, p_gate):
    error_meas = pauli_error([('X', p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") 
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x", "u1", "u2"]) 
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) 
        
    return noise_model

$\texttt{get_angles(a, n)}$ computes the rotation angle to be used in Fourier transform-like circuit for addition with $a$. $n$ denotes the number of bits used for representation. $a$ must be known classically.

In [0]:
def get_angles(a, n):
    bit = bin(abs(int(a)))[2:].zfill(n)
    theta = np.zeros(n)
    for i in range(n):
        for j in range(n-1-i, n):
            if bit[j] == '1':
                theta[i] += 2**(n-i-j-1)
    theta = theta * np.pi
    return theta

$\texttt{phi_add(circuit, q, a, n, c=[], inverse = False)}$ creates circuit to add(subtract, if $\texttt{inverse==True}$) $a$ to(from) $q$ modulo $2^n$ in Fourier Space. $n$ is the number of bits used for representation. $c$ is a (possibly empty) list of control bits. $a$ must be known classically. $q$ is a quantum register of $n$ bits.

In [0]:
def phi_add(circuit, q, a, n, c=[], inverse = False):
    theta = get_angles(a, n)    
    if(inverse):
        theta = -theta
    if not c:
        for i in range(n):
            circuit.u1(theta[i], q[i])    
    else:
        for i in range(n):
            circuit.mcu1(theta[i], c, q[i])    

$\texttt{phi_add_mod_N(circuit, q, a, n, N, c, aux, inverse = False)}$ creates circuit to add(subtract, if $\texttt{inverse==True}$) $a \in \mathbb{Z}^+$ to(from) $q$ mod $N$ in Fourier Space. $n$ is the number of bits used for representation. $c$ is a non-empty list of control bits. $a$ must be known classically. $q$ is a quantum register of $n+1$ bits. $q$ must be a non-negative integer less than $N$. $N$ must be a positive integer less than $2^n$. $aux$ is a single auxiliary qubit.

In [0]:
def phi_add_mod_N(circuit, q, a, n, N, c, aux, inverse = False):
    if not c :
        print('There should be at least one control qubit')
        return None
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    if(inverse):
        a = (-a) % N
    else: 
        a = a % N
        
    phi_add(circuit, q, a, n+1, c)
    phi_add(circuit, q, N, n+1, inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    circuit.cx(q[n], aux)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False)
    phi_add(circuit, q, N, n+1, [aux])
    phi_add(circuit, q, a, n+1, c, inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    circuit.x(q[n])
    circuit.cx(q[n], aux)
    circuit.x(q[n])
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False)
    phi_add(circuit, q, a, n+1, c)

$\texttt{multiply_mod_N(circuit, q, a, n, N, c, aux)}$ creates circuit to multiply $a$ to $q$ mod $N$ in usual bit representation. $n$ is the number of bits used for representation. $a \in \mathbb{Z}^+$ must be known classically. $q$ is a quantum register of $n$ bits. $q$ must be a non-negative integer less than $N$ if we want the output to be $q$ mod $N$ (instead of $q$) when $c$ is $0$. $N$ must be a positive integer less than $2^n$. $a$ and $N$ must be relatively prime. $c$ is the only control bit. $aux$ is an auxiliary quantum register of $n+2$ bits.

In [0]:
def multiply_mod_N(circuit, q, a, n, N, c, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    a = a % N
    a_inv = modinv(a, N)
    if(not a_inv):
        print('Inverse of a does not exist')
        return None
    pow_2_mod = np.zeros(n)
    pow_2_mod[0] = 1
    for i in range(1, n):
        pow_2_mod[i] = (2*pow_2_mod[i-1]) % N
        
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False)
    for i in range(n):
        phi_add_mod_N(circuit, aux, (pow_2_mod[i]*a) % N, n, N, [c, q[i]], aux[n+1])
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    
    for i in range(n):
        circuit.cswap(c, q[i], aux[i])

    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False)
    for i in reversed(range(n)):
        phi_add_mod_N(circuit, aux, (pow_2_mod[i]*a_inv) % N, n, N, [c, q[i]], aux[n+1], inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)

$\texttt{power_mod_N(circuit, q, r, a, m, n, N, aux)}$ creates circuit to compute $(r \cdot a^q )\text{ mod } N$ in the usual bit representation. $n$ is the number of bits used in the representation. $a$ must be known classically. $q$ and $r$ are quantum registers of $m$ and $n$ bits respectively. $N$ must be a positive integer less than $2^n$. $a$ and $N$ must be relatively prime. $aux$ is an auxiliary quantum register of $n+2$ bits. $r$ must be a non-negative integer less than $N$. However, if we know that $q$ is always positive, $r$ can be any non-negative integer less that $2^n$.

In [0]:
def power_mod_N(circuit, q, r, a, m, n, N, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    temp = a % N
    for i in range(m):
        multiply_mod_N(circuit, r, temp, n, N, q[i], aux)
        temp = (temp*temp) % N

$\texttt{discrete_log(a, b, N, n_shots=64)}$ returns  $t$ such that $a^t \equiv b \text{ mod } N$, with high probability, if such a $t$ exists. In this case, $t \in \mathbb{Z}_r$ where $r$ is the order of $a$ modulo $N$. Otherwise $t = \texttt{None}$.  $a, b \in \mathbb{Z}$ and $N \in \mathbb{Z}^{\geq 2}$ such that  $a$ and $N$ must be relatively prime.  This is the only function that uses the $\texttt{order()}$ function. One may replace its definition by Shor's order finding algorithm which works in polynomial time. This is not done here so as to focus only on the discrete log quantum algorithm.  $n\_shots$ is the maximum number of times the algorithm should be run to determine the answer.

In [0]:
def discrete_log(a, b, N, noise_model=None, n_shots=32):
    N = int(N)
    a = int(a) % N
    b = int(b) % N
    n_shots = int(n_shots)
    
    if(N<2 or gcd(a, N) != 1 or n_shots<1):
        print('Input error')
        return None
    if(gcd(b, N) != 1):
        return None
    
    r = int(order(a, N))
    if(r==1):
        if(b==1):
            return 0
        else:
            return None
    m = int(np.ceil(np.log2(N+1)))
    n = int(np.ceil(np.log2(r+1))) + 1
    q = QuantumRegister(2*(m+n+1), 'q')
    c = ClassicalRegister(2*n, 'c')
    circuit = QuantumCircuit(q, c)
    
    for i in range(2*n):
        circuit.h(q[i])
    circuit.x(q[2*n])
    power_mod_N(circuit, q[:n], q[2*n:2*n+m], a, n, m, N, q[2*n+m:2*n+2*m+2])
    power_mod_N(circuit, q[n:2*n], q[2*n:2*n+m], b, n, m, N, q[2*n+m:2*n+2*m+2])
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n))], inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n, 2*n))], inverse=True)
    circuit.measure(q[:2*n], c[:2*n])
    
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, noise_model=noise_model, shots=n_shots)
    result = job.result()
    counts = result.get_counts(circuit)
    M = len(counts)
     
    k = []
    kt_mod_r = []
    for key in counts:
        x = int(key[-n:], 2) 
        y = int(key[-2*n:-n], 2) 
        k.append(round((x*r)/(2**n)))
        kt_mod_r.append(round((y*r)/(2**n)))

    for j in range(M):
        for i in range(j+1):
            if(gcd(k[i], k[j]) == 1):
                _, v1, v2 = egcd(k[i], k[j])
                t = (kt_mod_r[i]*v1 + kt_mod_r[j]*v2)%r
                if(mod_exp(a, t, N) == b):
                    return t
    return None

Change the values of $a, b$ and $N$ in the following cell. Also, select the noise model by setting the arguments to the $\texttt{get_noise()}$ function. See the cell for $\texttt{get_noise()}$ for more details on noise parameters.

In [0]:
a = 6
b = 30
N = 31
n_shots = 32
#noise_model = get_noise(0.01, 0.01)
noise_model = None
s_time = time()
print('t =', discrete_log(a, b, N, noise_model, n_shots))
print('Time:', round(time() - s_time), 's') 

t = 3
Time: 197 s
